**販売確率の高い上位15の商品ラインナップ**

In [1]:
# coding:utf-8
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import ensemble, metrics
from datetime import datetime as dt
from sklearn.preprocessing import LabelEncoder


# pandas
import pandas as pd
#現在の最大表示行数の出力
pd.get_option("display.max_rows")
#最大表示行数の指定（ここでは50行を指定）
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_colwidth", 1000)

# LightGBM
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 交差検証
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

# AUC
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

/Users/toshiki/py37env/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
purchase_df = pd.read_csv('../input/purchase_record.csv')

In [3]:
purchase_df

,user_id,purchase_id,date,product_id,parts_1,parts_2,parts_3,parts_4,parts_5,parts_6,parts_7,parts_8,parts_9,purchase
0,uid_100,pid_0000000000,2017-09-27,product 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,uid_100,pid_0000000001,2017-09-27,product 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,uid_100002,pid_0000000002,2017-11-23,product 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,uid_100002,pid_0000000003,2017-11-23,product 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,uid_100003,pid_0000000004,2018-02-07,product 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1803569,uid_99994,pid_0001803569,2018-06-16,product 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1803570,uid_99994,pid_0001803570,2018-03-30,product 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1803571,uid_99994,pid_0001803571,2018-01-19,product 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1803572,uid_99994,pid_0001803572,2018-05-02,product 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [4]:
purchase_df = purchase_df[purchase_df['purchase'] == 1]

In [5]:
purchase_df

,user_id,purchase_id,date,product_id,parts_1,parts_2,parts_3,parts_4,parts_5,parts_6,parts_7,parts_8,parts_9,purchase
2,uid_100002,pid_0000000002,2017-11-23,product 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,uid_100002,pid_0000000003,2017-11-23,product 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,uid_100003,pid_0000000004,2018-02-07,product 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
5,uid_100003,pid_0000000005,2018-06-28,product 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
6,uid_100003,pid_0000000006,2017-08-10,product 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1803569,uid_99994,pid_0001803569,2018-06-16,product 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1803570,uid_99994,pid_0001803570,2018-03-30,product 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1803571,uid_99994,pid_0001803571,2018-01-19,product 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1803572,uid_99994,pid_0001803572,2018-05-02,product 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [6]:
# 不要列削除　
purchase_df = purchase_df.drop('purchase_id', axis=1)
purchase_df = purchase_df.drop('user_id', axis=1)
purchase_df = purchase_df.drop('date', axis=1)
purchase_df = purchase_df.drop('purchase', axis=1)

In [7]:
purchase_df.head(1000)

,product_id,parts_1,parts_2,parts_3,parts_4,parts_5,parts_6,parts_7,parts_8,parts_9
2,product 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,product 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,product 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,product 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,product 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,product 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,product 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,product 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,product 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,product 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# group = purchase_df.assign(freq=0)
# freq = group.groupby(group.columns.tolist()).size().reset_index(name='count')
# group2 = freq.sort_values('count', ascending=False)
# group2.head(1000)

In [9]:
purchase_df

,product_id,parts_1,parts_2,parts_3,parts_4,parts_5,parts_6,parts_7,parts_8,parts_9
2,product 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,product 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,product 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,product 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,product 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1803569,product 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1803570,product 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1803571,product 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1803572,product 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
join_df = purchase_df.replace(np.nan,'xx x', regex=True)
join_df['join'] = purchase_df['product_id'] + '-' + purchase_df['parts_1'] + '-' + purchase_df['parts_2'] + '-' + purchase_df['parts_3'] + '-'+ purchase_df['parts_4'] + '-' + purchase_df['parts_5'] + '-' + purchase_df['parts_6'] + '-' + purchase_df['parts_7'] + '-' +purchase_df['parts_8'] + '-' +purchase_df['parts_9']

In [12]:
grouped_product_id = join_df.groupby("join").size().reset_index(name='product_id_counts')

In [13]:
grouped_product_id.sort_values('product_id_counts', ascending=False).head(1000)

,join,product_id_counts
2578,product 2-xx x-xx x-xx x-xx x-xx x-xx x-xx x-xx x-xx x,415995
2581,product 5-xx x-xx x-xx x-xx x-xx x-xx x-xx x-xx x-xx x,225313
2579,product 3-xx x-xx x-xx x-xx x-xx x-xx x-xx x-xx x-xx x,89706
2580,product 4-xx x-xx x-xx x-xx x-xx x-xx x-xx x-xx x-xx x,84727
1370,product 1-id 2-id 5-id 5-id 3-id 6-id 1-id 1-id 4-id 8,3247
2582,product 6-xx x-xx x-xx x-xx x-xx x-xx x-xx x-xx x-xx x,2046
1757,product 1-id 3-id 5-id 1-id 1-id 6-id 3-id 3-id 6-id 8,1456
1346,product 1-id 2-id 5-id 3-id 3-id 6-id 2-id 2-id 4-id 8,1115
0,product 1-id 1-id 1-id 1-id 1-id 1-id 1-id 1-id 1-id 1,1049
2583,product 7-xx x-xx x-xx x-xx x-xx x-xx x-xx x-xx x-xx x,1042
